In [1]:
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = "http://www.krei.re.kr:18181/chart/main_chart/index_grid_total/kind/S/sdate/1972-01-01/edate/2022-11-11/mode/1"

html = requests.get(url)
soup = BeautifulSoup(html.text, "html.parser")
# soup.prettify()

In [3]:
json_obj = json.loads(html.content)
json_obj.keys()

dict_keys(['columns', 'data'])

In [4]:
# Explore What is in columns and data values
json_obj["columns"]

[{'title': 'Date', 'data': 'date'},
 {'title': 'Type', 'data': 'sy'},
 {'title': 'Open', 'data': 'op'},
 {'title': 'Close', 'data': 'cl'},
 {'title': 'Settlement', 'data': 'se'},
 {'title': 'Volume', 'data': 'vo'}]

In [5]:
df = pd.DataFrame.from_dict(json_obj['data'])

In [6]:
df["sy"].unique()

array(['Wheat', 'Soybeans', 'Corn'], dtype=object)

In [7]:
df.dtypes

date    object
sy      object
op      object
cl      object
se      object
vo      object
dtype: object

In [8]:
df = df.astype({'op':"float64","cl":"float64","se":"float64","vo":"float64"})
df["date"] = pd.to_datetime(df["date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38420 entries, 0 to 38419
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    38420 non-null  datetime64[ns]
 1   sy      38420 non-null  object        
 2   op      38420 non-null  float64       
 3   cl      38420 non-null  float64       
 4   se      38420 non-null  float64       
 5   vo      38420 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.8+ MB


In [9]:
df.rename(columns={'op': 'open', 'cl': 'close', 'se':'settle', 'vo':'volume', 'sy':'product'}, inplace=True)

In [10]:
df.to_csv("./data/df.csv", encoding="utf-8-sig", index=False)

In [11]:
import plotly.express as px

fig = px.line(df, 
              x= 'date',
              y= 'settle',
              color = 'product')

fig.show()